In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [39]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_curve,
    roc_auc_score,
    f1_score,
    make_scorer,
)
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
import os

print(os.getcwd())

print(os.listdir(os.getcwd()))

C:\Users\jlim7\OneDrive\Documents\GitHub\Thinkful_work\Thinkful\Checkpoints
['.ipynb_checkpoints', '1067_1925_bundle_archive.zip', 'Analyzing Model Performance.ipynb', 'Beyond AB Tests Checkpoint.ipynb', 'Building and Analyzing Performance of Multiclass Classifiers.ipynb', 'Case Study Email checkpoint.ipynb', 'Challenge.ipynb', 'Classification Using Linear Classifiers .ipynb', 'Data Cleaning II.ipynb', 'Data Cleaning III.ipynb', 'Data Exploration I.ipynb', 'Data Exploration II.ipynb', 'Evaluating Performance Checkpoint.ipynb', 'Feature Engineering I.ipynb', 'Getting SQL Data into Python.ipynb', 'Interpreting Estimated Coefficients Checkpoint.ipynb', 'Linear Regression Assumptions.ipynb', 'Linear Regression with OLS Assumptions.ipynb', 'Linear Regression with OLS.ipynb', 'Making Predictions Checkpoint.ipynb', 'Overfitting and Regularization Checkpoint.ipynb', 'Performing a T-Test.ipynb', 'Random Forest Challenge.ipynb', 'SQL_DrillsII.sql', 'SQL_DrillsIII.sql', 'Supervised Learning Chall

<IPython.core.display.Javascript object>

In [12]:
def print_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")

<IPython.core.display.Javascript object>

In [3]:
pd.set_option("display.max_columns", None)

<IPython.core.display.Javascript object>

In [4]:
# Education
# 1 'Below College'
# 2 'College'
# 3 'Bachelor'
# 4 'Master'
# 5 'Doctor'

# EnvironmentSatisfaction
# 1 'Low'
# 2 'Medium'
# 3 'High'
# 4 'Very High'

# JobInvolvement
# 1 'Low'
# 2 'Medium'
# 3 'High'
# 4 'Very High'

# JobSatisfaction
# 1 'Low'
# 2 'Medium'
# 3 'High'
# 4 'Very High'

# PerformanceRating
# 1 'Low'
# 2 'Good'
# 3 'Excellent'
# 4 'Outstanding'

# RelationshipSatisfaction
# 1 'Low'
# 2 'Medium'
# 3 'High'
# 4 'Very High'

# WorkLifeBalance
# 1 'Bad'
# 2 'Good'
# 3 'Better'
# 4 'Best'

<IPython.core.display.Javascript object>

In [5]:
hr_df = pd.read_csv("1067_1925_bundle_archive.zip")
hr_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


<IPython.core.display.Javascript object>

In [6]:
hr_df["BusinessTravel"].value_counts()

Travel_Rarely        1043
Travel_Frequently     277
Non-Travel            150
Name: BusinessTravel, dtype: int64

<IPython.core.display.Javascript object>

In [7]:
hr_df["Department"].value_counts()

Research & Development    961
Sales                     446
Human Resources            63
Name: Department, dtype: int64

<IPython.core.display.Javascript object>

In [8]:
hr_df["EmployeeCount"].value_counts()

1    1470
Name: EmployeeCount, dtype: int64

<IPython.core.display.Javascript object>

In [9]:
hr_df = pd.get_dummies(
    hr_df,
    columns=[
        "Attrition",
        "BusinessTravel",
        "Department",
        "Gender",
        "Over18",
        "OverTime",
    ],
    drop_first=True,
)

<IPython.core.display.Javascript object>

In [10]:
hr_df = hr_df.drop(
    columns=[
        "EducationField",
        "EmployeeCount",
        "EmployeeNumber",
        "JobRole",
        "MaritalStatus",
    ]
)

<IPython.core.display.Javascript object>

In [11]:
hr_df.head()

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_Yes,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Research & Development,Department_Sales,Gender_Male,OverTime_Yes
0,41,1102,1,2,2,94,3,2,4,5993,19479,8,11,3,1,80,0,8,0,1,6,4,0,5,1,0,1,0,1,0,1
1,49,279,8,1,3,61,2,2,2,5130,24907,1,23,4,4,80,1,10,3,3,10,7,1,7,0,1,0,1,0,1,0
2,37,1373,2,2,4,92,2,1,3,2090,2396,6,15,3,2,80,0,7,3,3,0,0,0,0,1,0,1,1,0,1,1
3,33,1392,3,4,4,56,3,1,3,2909,23159,1,11,3,3,80,0,8,3,3,8,7,3,0,0,1,0,1,0,0,1
4,27,591,2,1,1,40,3,1,2,3468,16632,9,12,3,4,80,1,6,3,3,2,2,2,2,0,0,1,1,0,1,0


<IPython.core.display.Javascript object>

In [13]:
hr_df.dtypes

Age                                  int64
DailyRate                            int64
DistanceFromHome                     int64
Education                            int64
EnvironmentSatisfaction              int64
HourlyRate                           int64
JobInvolvement                       int64
JobLevel                             int64
JobSatisfaction                      int64
MonthlyIncome                        int64
MonthlyRate                          int64
NumCompaniesWorked                   int64
PercentSalaryHike                    int64
PerformanceRating                    int64
RelationshipSatisfaction             int64
StandardHours                        int64
StockOptionLevel                     int64
TotalWorkingYears                    int64
TrainingTimesLastYear                int64
WorkLifeBalance                      int64
YearsAtCompany                       int64
YearsInCurrentRole                   int64
YearsSinceLastPromotion              int64
YearsWithCu

<IPython.core.display.Javascript object>

In [15]:
X = hr_df.drop(columns=["Attrition_Yes"])
y = hr_df["Attrition_Yes"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

<IPython.core.display.Javascript object>

In [16]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

<IPython.core.display.Javascript object>

In [17]:
model.score(X_train, y_train)

1.0

<IPython.core.display.Javascript object>

In [18]:
model.score(X_test, y_test)

0.8639455782312925

<IPython.core.display.Javascript object>

In [23]:
print_vif(X_train)

VIF results
-------------------------------
Age                                    2.080015
DailyRate                              1.019186
DistanceFromHome                       1.015425
Education                              1.060047
EnvironmentSatisfaction                1.021481
HourlyRate                             1.025479
JobInvolvement                         1.026882
JobLevel                              12.144255
JobSatisfaction                        1.034372
MonthlyIncome                         11.304720
MonthlyRate                            1.021991
NumCompaniesWorked                     1.278217
PercentSalaryHike                      2.561606
PerformanceRating                      2.561787
RelationshipSatisfaction               1.025247
StandardHours                        245.227551
StockOptionLevel                       1.017470
TotalWorkingYears                      4.763210
TrainingTimesLastYear                  1.028719
WorkLifeBalance                        1.030

<IPython.core.display.Javascript object>

In [28]:
X = hr_df.drop(columns=["Attrition_Yes", "StandardHours", "JobLevel"])
y = hr_df["Attrition_Yes"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

<IPython.core.display.Javascript object>

In [29]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

<IPython.core.display.Javascript object>

In [30]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

1.0
0.8843537414965986


<IPython.core.display.Javascript object>

In [31]:
print_vif(X_train)

VIF results
-------------------------------
const                                238.010020
Age                                    2.058201
DailyRate                              1.026760
DistanceFromHome                       1.013843
Education                              1.066293
EnvironmentSatisfaction                1.023566
HourlyRate                             1.022016
JobInvolvement                         1.022683
JobSatisfaction                        1.019645
MonthlyIncome                          2.608077
MonthlyRate                            1.018651
NumCompaniesWorked                     1.276004
PercentSalaryHike                      2.647864
PerformanceRating                      2.663127
RelationshipSatisfaction               1.029503
StockOptionLevel                       1.024018
TotalWorkingYears                      4.758428
TrainingTimesLastYear                  1.025479
WorkLifeBalance                        1.021041
YearsAtCompany                         4.600

<IPython.core.display.Javascript object>

In [35]:
grid = {"max_depth": [3, 5, 8, 10], "n_estimators": [100, 150, 200, 300]}

model = GridSearchCV(
    RandomForestClassifier(),
    param_grid=grid,
    # If accuracy isnt the main metric you care about
    # you can have CV use a different one
    scoring=make_scorer(roc_auc_score),
    #         scoring=make_scorer(f1_score),
    cv=3,
    verbose=1,
)
model.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:   12.5s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

<IPython.core.display.Javascript object>

In [36]:
model.best_params_

{'max_depth': 10, 'n_estimators': 300}

<IPython.core.display.Javascript object>

In [37]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.946969696969697
0.5730015082956259


<IPython.core.display.Javascript object>

In [45]:
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[253,   2],
       [ 33,   6]], dtype=int64)

<IPython.core.display.Javascript object>

In [46]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.99      0.94       255
           1       0.75      0.15      0.26        39

    accuracy                           0.88       294
   macro avg       0.82      0.57      0.60       294
weighted avg       0.87      0.88      0.85       294



<IPython.core.display.Javascript object>

In [47]:
importance_df = pd.DataFrame(
    {"feat": X_train.columns, "importance": model.best_estimator_.feature_importances_}
)

importance_df = importance_df.sort_values("importance", ascending=False)
importance_df

,feat,importance
8,MonthlyIncome,0.090015
0,Age,0.072853
27,OverTime_Yes,0.059703
1,DailyRate,0.056072
9,MonthlyRate,0.054388
5,HourlyRate,0.053997
15,TotalWorkingYears,0.053611
2,DistanceFromHome,0.048429
18,YearsAtCompany,0.048245
14,StockOptionLevel,0.043622


<IPython.core.display.Javascript object>